# Module 03: Exploring NIXL (NVIDIA Inference Xfer Library)

> **Goal**: Understand how NIXL enables fast GPU-to-GPU KV cache transfers in disaggregated serving.

---

## What is NIXL?

**NIXL** (NVIDIA Inference Xfer Library) is a high-performance data transfer library designed for AI inference. It provides:

- **Direct GPU-to-GPU transfers** via RDMA (Remote Direct Memory Access)
- **Unified API** across different memory types (GPU, CPU, storage)
- **Modular backends** (UCX, GDS, etc.)

### Why NIXL Matters for Disaggregated Serving

In Module 02, we saw prefill and decode workers on separate GPUs. The KV cache must move between them:

```
┌─────────────────┐                    ┌─────────────────┐
│  Prefill Worker │                    │  Decode Worker  │
│     (GPU 0)     │                    │     (GPU 1)     │
│                 │                    │                 │
│  KV Cache       │═══════NIXL════════▶│  KV Cache       │
│  (VRAM)         │   Direct Transfer  │  (VRAM)         │
│                 │   ~10-50 GB/s      │                 │
└─────────────────┘                    └─────────────────┘
```

Without NIXL, you'd need to:
1. Copy KV cache from GPU 0 → CPU RAM
2. Send over network/IPC
3. Copy from CPU RAM → GPU 1

NIXL bypasses the CPU entirely using RDMA, achieving much higher throughput.

### NIXL Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                      Application Layer                       │
│                    (Dynamo, vLLM, etc.)                      │
├─────────────────────────────────────────────────────────────┤
│                        NIXL API                              │
│         nixlAgent, registerMem, createXferReq, etc.         │
├─────────────────────────────────────────────────────────────┤
│                    Backend Plugins                           │
│  ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────────────┐ │
│  │   UCX   │  │   GDS   │  │  POSIX  │  │  Object Store   │ │
│  │ (RDMA)  │  │ (NVMe)  │  │ (Files) │  │    (S3/GCS)     │ │
│  └─────────┘  └─────────┘  └─────────┘  └─────────────────┘ │
├─────────────────────────────────────────────────────────────┤
│                    Memory Segments                           │
│        VRAM_SEG (GPU)  |  DRAM_SEG (CPU)  |  FILE_SEG       │
└─────────────────────────────────────────────────────────────┘
```

---

## Step 1: Install and Import NIXL

In [ ]:
# Install NIXL if not already installed
!uv pip install nixl

import nixl
print(f"NIXL installed at: {nixl.__file__}")
print(f"\nNIXL version info:")
!uv pip show nixl | grep -E "^(Name|Version|Summary)"

---

## Step 2: Explore NIXL API

Let's look at the main components of NIXL.

In [ ]:
import nixl

print("=" * 60)
print("NIXL API OVERVIEW")
print("=" * 60)

# Memory segment types
print("\n📦 Memory Segment Types:")
segments = ['VRAM_SEG', 'DRAM_SEG', 'FILE_SEG', 'BLK_SEG', 'OBJ_SEG']
for seg in segments:
    if hasattr(nixl, seg):
        val = getattr(nixl, seg)
        desc = {
            'VRAM_SEG': 'GPU memory (CUDA)',
            'DRAM_SEG': 'CPU memory (RAM)',
            'FILE_SEG': 'File-based storage',
            'BLK_SEG': 'Block storage',
            'OBJ_SEG': 'Object storage (S3, GCS)'
        }.get(seg, '')
        print(f"  {seg} = {val}  # {desc}")

# Transfer operations
print("\n🔄 Transfer Operations:")
print(f"  NIXL_READ  = {nixl.NIXL_READ}   # Pull data from remote")
print(f"  NIXL_WRITE = {nixl.NIXL_WRITE}  # Push data to remote")

# Status codes
print("\n✓ Status Codes:")
status_codes = ['NIXL_SUCCESS', 'NIXL_IN_PROG', 'NIXL_ERR_NOT_FOUND', 'NIXL_ERR_BACKEND']
for code in status_codes:
    if hasattr(nixl, code):
        print(f"  {code} = {getattr(nixl, code)}")

---

## Step 3: Understand the nixlAgent

The `nixlAgent` is the core class in NIXL. Each process that wants to transfer data creates an agent.

In [ ]:
from nixl import nixlAgent, nixlAgentConfig

print("=" * 60)
print("nixlAgent - CORE TRANSFER CLASS")
print("=" * 60)

# Show agent methods
agent_methods = [m for m in dir(nixlAgent) if not m.startswith('_')]
print(f"\nAgent has {len(agent_methods)} methods:\n")

# Group by function
method_groups = {
    'Memory': ['registerMem', 'deregisterMem', 'queryMem'],
    'Transfer': ['createXferReq', 'makeXferReq', 'postXferReq', 'releaseXferReq'],
    'Status': ['getXferStatus', 'getXferTelemetry', 'estimateXferCost'],
    'Metadata': ['getLocalMD', 'fetchRemoteMD', 'loadRemoteMD', 'sendLocalMD'],
    'Notification': ['genNotif', 'getNotifs'],
    'Backend': ['createBackend', 'getAvailPlugins', 'getBackendParams'],
    'Connection': ['makeConnection', 'checkRemoteMD'],
}

for group, methods in method_groups.items():
    print(f"📌 {group}:")
    for m in methods:
        if m in agent_methods:
            print(f"     • {m}()")
    print()

---

## Step 4: Create a NIXL Agent

Let's create an agent and explore its capabilities.

In [ ]:
from nixl import nixlAgent, nixlAgentConfig
import uuid

print("=" * 60)
print("CREATING A NIXL AGENT")
print("=" * 60)

# Create agent configuration
config = nixlAgentConfig()
agent_name = f"notebook-agent-{uuid.uuid4().hex[:8]}"

# Create the agent
agent = nixlAgent(agent_name, config)
print(f"\n✓ Created agent: {agent_name}")

# Check available plugins/backends
plugins = agent.getAvailPlugins()
print(f"\n📦 Available backends/plugins:")
for plugin in plugins:
    print(f"   • {plugin}")
    try:
        params, required = agent.getPluginParams(plugin)
        if params:
            print(f"     Parameters: {params}")
    except:
        pass

---

## Step 5: Create a UCX Backend

UCX (Unified Communication X) is the primary backend for GPU-to-GPU transfers. It supports:
- **RDMA** (InfiniBand, RoCE)
- **Shared memory** (same node)
- **TCP** (fallback)

In [ ]:
print("=" * 60)
print("CREATING UCX BACKEND")
print("=" * 60)

# Create UCX backend with default options
ucx_options = {}  # Use defaults

try:
    backend_handle = agent.createBackend("UCX", ucx_options)
    print(f"\n✓ UCX backend created, handle: {backend_handle}")
    
    # Get backend parameters
    params, supported = agent.getBackendParams(backend_handle)
    print(f"\n📋 UCX Backend Configuration:")
    for key, value in params.items():
        print(f"   {key}: {value}")
        
except Exception as e:
    print(f"\n⚠️  Could not create UCX backend: {e}")
    print("   This may require specific UCX libraries or RDMA hardware.")

---

## Step 6: Register GPU Memory

Before transferring data, memory regions must be registered with NIXL. This allows the backend to set up RDMA mappings.

In [ ]:
import torch
from nixl import VRAM_SEG, DRAM_SEG

print("=" * 60)
print("REGISTERING GPU MEMORY")
print("=" * 60)

# Allocate some GPU memory (simulating a KV cache)
if torch.cuda.is_available():
    # Create a tensor on GPU (like a KV cache layer)
    kv_cache_size = (32, 128, 64)  # (batch, seq_len, hidden_dim)
    gpu_tensor = torch.randn(kv_cache_size, device='cuda:0', dtype=torch.float16)
    
    print(f"\n📊 Created GPU tensor:")
    print(f"   Shape: {gpu_tensor.shape}")
    print(f"   Size: {gpu_tensor.numel() * 2 / 1024:.2f} KB")  # float16 = 2 bytes
    print(f"   Device: {gpu_tensor.device}")
    print(f"   Data ptr: {hex(gpu_tensor.data_ptr())}")
    
    # Create a descriptor list for registration
    # Note: This shows the concept - actual registration requires proper setup
    print(f"\n📝 Memory Registration Concept:")
    print(f"   1. Get tensor data pointer: {hex(gpu_tensor.data_ptr())}")
    print(f"   2. Calculate size in bytes: {gpu_tensor.numel() * gpu_tensor.element_size()}")
    print(f"   3. Specify segment type: VRAM_SEG = {VRAM_SEG}")
    print(f"   4. Register with backend for RDMA access")
else:
    print("⚠️  No GPU available for memory registration demo")

---

## Step 7: Understanding the Transfer Flow

Here's how a KV cache transfer works in disaggregated serving:

In [ ]:
print("=" * 60)
print("NIXL TRANSFER FLOW")
print("=" * 60)

flow = """
┌─────────────────────────────────────────────────────────────────┐
│                    NIXL KV Cache Transfer                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  PREFILL WORKER (GPU 0)              DECODE WORKER (GPU 1)       │
│  ════════════════════                ═══════════════════════     │
│                                                                  │
│  1. Create nixlAgent                 1. Create nixlAgent         │
│     agent_p = nixlAgent(...)            agent_d = nixlAgent(...) │
│                                                                  │
│  2. Create UCX backend               2. Create UCX backend       │
│     backend = createBackend("UCX")      backend = createBackend()│
│                                                                  │
│  3. Register KV cache memory         3. Register KV cache memory │
│     registerMem(kv_cache_ptr)           registerMem(kv_cache_ptr)│
│                                                                  │
│  4. Export metadata ──────────────▶  4. Fetch remote metadata    │
│     md = getLocalMD()                   fetchRemoteMD(agent_p)   │
│     sendLocalMD()                                                │
│                                                                  │
│  5. (After prefill completes)        5. Create transfer request  │
│     Generate KV cache data              req = createXferReq(     │
│                                           NIXL_READ,             │
│                                           local_kv, remote_kv)   │
│                                                                  │
│  6. Wait for read request            6. Post transfer            │
│     (RDMA read from remote)             postXferReq(req)         │
│                                                                  │
│  7. ◀═══════ RDMA DATA ══════════▶   7. Poll for completion      │
│     (Direct GPU-to-GPU)                 while IN_PROG:           │
│                                           getXferStatus(req)     │
│                                                                  │
│  8. Send notification                8. Receive notification     │
│     genNotif("kv_ready")                getNotifs() → "kv_ready" │
│                                                                  │
│                                      9. Use KV cache for decode  │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
"""
print(flow)

---

## Step 8: See NIXL in Action (Dynamo Logs)

Let's look at NIXL activity in the Dynamo worker logs from Module 02.

In [ ]:
import os

print("=" * 60)
print("NIXL IN DYNAMO LOGS")
print("=" * 60)

log_files = ['/tmp/dynamo_prefill.log', '/tmp/dynamo_decode.log']

for log_file in log_files:
    if os.path.exists(log_file):
        worker_type = "PREFILL" if "prefill" in log_file else "DECODE"
        print(f"\n{'🔵' if worker_type == 'PREFILL' else '🟢'} {worker_type} Worker NIXL activity:")
        print("-" * 50)
        
        with open(log_file, 'r') as f:
            lines = f.readlines()
            nixl_lines = [l.strip() for l in lines if 'NIXL' in l.upper() or 'nixl' in l.lower()]
            
            if nixl_lines:
                for line in nixl_lines[:10]:  # Show first 10 NIXL-related lines
                    # Clean up the line for display
                    if len(line) > 100:
                        line = line[:100] + "..."
                    print(f"  {line}")
            else:
                print("  No NIXL log entries found")
    else:
        print(f"\n⚠️  Log file not found: {log_file}")
        print("   Run Module 02 first to generate logs")

---

## Step 9: NIXL Performance Characteristics

Understanding when NIXL provides benefits.

In [ ]:
print("=" * 60)
print("NIXL PERFORMANCE CHARACTERISTICS")
print("=" * 60)

print("""
┌────────────────────────────────────────────────────────────────┐
│                    Transfer Method Comparison                   │
├──────────────────┬─────────────────┬───────────────────────────┤
│ Method           │ Bandwidth       │ Latency                   │
├──────────────────┼─────────────────┼───────────────────────────┤
│ NIXL + NVLink    │ ~600 GB/s       │ ~1-2 μs (same node)       │
│ NIXL + RDMA      │ ~25-100 GB/s    │ ~2-5 μs (cross node)      │
│ NIXL + PCIe      │ ~25-32 GB/s     │ ~5-10 μs                  │
│ CPU Copy         │ ~10-20 GB/s     │ ~50-100 μs (memcpy)       │
│ TCP/IP           │ ~1-10 GB/s      │ ~100+ μs                  │
└──────────────────┴─────────────────┴───────────────────────────┘

Key Benefits of NIXL:
─────────────────────
✓ Zero-copy: Data goes directly GPU→GPU without CPU involvement
✓ Async: Transfers happen in background while GPU computes
✓ Batched: Multiple memory regions can be transferred together
✓ Unified: Same API for local (NVLink) and remote (RDMA) transfers

When NIXL Matters Most:
───────────────────────
• Large KV caches (longer context = more data to transfer)
• High request throughput (many transfers per second)
• Multi-node deployments (cross-machine transfers)
• Mixed prefill/decode ratios (frequent KV cache handoffs)
""")

# Calculate example transfer times
kv_cache_sizes_gb = [1, 4, 16, 64]
print("\n📊 Example KV Cache Transfer Times:")
print("-" * 50)
print(f"{'KV Cache Size':<15} {'RDMA (50GB/s)':<18} {'CPU Copy (15GB/s)':<18}")
print("-" * 50)
for size in kv_cache_sizes_gb:
    rdma_time = size / 50 * 1000  # ms
    cpu_time = size / 15 * 1000   # ms
    print(f"{size} GB{'':<10} {rdma_time:.1f} ms{'':<12} {cpu_time:.1f} ms")

---

## Step 10: How Dynamo Uses NIXL

Let's trace how Dynamo integrates NIXL for disaggregated serving.

In [ ]:
print("=" * 60)
print("DYNAMO + NIXL INTEGRATION")
print("=" * 60)

print("""
Dynamo wraps NIXL in a higher-level abstraction:

┌─────────────────────────────────────────────────────────────────┐
│                     Dynamo Architecture                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │                  dynamo.sglang Worker                    │    │
│  │  --disaggregation-mode prefill/decode                    │    │
│  │  --disaggregation-transfer-backend nixl                  │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                   │
│                              ▼                                   │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │              dynamo.nixl_connect Module                  │    │
│  │  • Manages NIXL agent lifecycle                          │    │
│  │  • Handles metadata exchange via bootstrap server        │    │
│  │  • Coordinates KV cache transfers                        │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                   │
│                              ▼                                   │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │                    NIXL Library                          │    │
│  │  nixlAgent → UCX Backend → RDMA/NVLink Hardware         │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘

Key Dynamo Flags:
─────────────────
  --disaggregation-transfer-backend nixl   # Use NIXL for transfers
  --disaggregation-transfer-backend mooncake  # Alternative backend
  --host 0.0.0.0                           # Required for bootstrap server
""")

---

## Key Takeaways

1. **NIXL** is NVIDIA's library for high-performance point-to-point data transfers
2. **nixlAgent** is the core class - each worker creates one to participate in transfers
3. **UCX backend** provides RDMA support for GPU-direct transfers
4. **Memory registration** is required before transfers (enables RDMA mappings)
5. **Metadata exchange** allows agents to discover each other's memory layout
6. **Async transfers** let GPU continue computing while data moves
7. **Dynamo wraps NIXL** in the `--disaggregation-transfer-backend nixl` flag

### Resources

| Resource | URL |
|----------|-----|
| NIXL GitHub | https://github.com/ai-dynamo/nixl |
| NIXL Docs | https://github.com/ai-dynamo/nixl/blob/main/docs/nixl.md |
| vLLM NIXL Guide | https://docs.vllm.ai/en/stable/features/nixl_connector_usage/ |

---

## Cleanup

In [ ]:
# Clean up GPU memory
import gc
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✓ GPU memory cleared")

# The agent will be garbage collected automatically
print("✓ Cleanup complete")